![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_DistilBertForMultipleChoice.ipynb)

# Import OpenVINO DistilBertForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting DistilBertForMultipleChoice  models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for DistilBertForMultipleChoice from DistilBertForMultipleChoice  and they have to be in `Multiple Choice` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.41.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers==4.41.2
!pip install -q --upgrade openvino==2024.1
!pip install -q --upgrade optimum-intel==1.17.0
!pip install -q --upgrade onnx==1.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- Normally, to load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. However, ForMultipleChoice is not yet available so we will use `openvino.convert_model()` after exporting ONNX model
- We'll use [irfanamal/bert_multiple_choice](https://huggingface.co/irfanamal/bert_multiple_choice) model from HuggingFace as an example
- We also need the `vocab.txt` saved from `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
!pip install --upgrade pip
!pip install -q --upgrade transformers[onnx] optimum openvino==2024.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 151.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.2 which is incompatible.
optimum-intel 1.17.0 requires transformers<4.42.0,>=4.36.0, but you have transformers 4.47.1 which is incompatible.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4

In [ ]:
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "daze-unlv/distilbert-distilbert-base-uncased"
ONNX_MODEL_PATH = f"onnx_models/distilbert_multiple_choice"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL_PATH)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
!ls -lh {ONNX_MODEL_PATH}

total 257M
-rw-r--r-- 1 root root  579 Dec 31 22:03 config.json
-rw-r--r-- 1 root root 256M Dec 31 22:03 model.onnx
-rw-r--r-- 1 root root  695 Dec 31 22:03 special_tokens_map.json
-rw-r--r-- 1 root root 1.3K Dec 31 22:03 tokenizer_config.json
-rw-r--r-- 1 root root 695K Dec 31 22:03 tokenizer.json
-rw-r--r-- 1 root root 227K Dec 31 22:03 vocab.txt


In [ ]:
import openvino as ov

# ov.convert_model returns an openvino.runtime.Model object
ov_model = ov.convert_model(f"{ONNX_MODEL_PATH}/model.onnx")

OPENVINO_MODEL_PATH = "./openvino_models"
# then model can be serialized to *.xml & *.bin files
ov.save_model(ov_model, f"{OPENVINO_MODEL_PATH}/openvino_model.xml")

In [ ]:
!ls -lh {OPENVINO_MODEL_PATH}

total 129M
-rw-r--r-- 1 root root 128M Dec 31 22:03 openvino_model.bin
-rw-r--r-- 1 root root 344K Dec 31 22:03 openvino_model.xml


In [ ]:
!mkdir {OPENVINO_MODEL_PATH}/assets

In [ ]:
!mv {ONNX_MODEL_PATH}/vocab.txt {OPENVINO_MODEL_PATH}/assets

Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -lR {OPENVINO_MODEL_PATH}

./openvino_models:
total 131124
drwxr-xr-x 2 root root      4096 Dec 31 22:03 assets
-rw-r--r-- 1 root root 133912812 Dec 31 22:03 openvino_model.bin
-rw-r--r-- 1 root root    352049 Dec 31 22:03 openvino_model.xml

./openvino_models/assets:
total 228
-rw-r--r-- 1 root root 231508 Dec 31 22:03 vocab.txt


## Import and Save DistilBertForMultipleChoice in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

Apache Spark version: 3.5.3


- Let's use `loadSavedModel` functon in `DistilBertForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForMultipleChoice` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *


distilBertMultpleChoiceClassifier = DistilBertForMultipleChoice.loadSavedModel(
     f"{OPENVINO_MODEL_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
distilBertMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {ONNX_MODEL_PATH}
!rm -rf {OPENVINO_MODEL_PATH}

Awesome 😎  !

This is your`DistilBertForMultipleChoice` model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 131148
-rw-r--r-- 1 root root 134285629 Dec 31 22:07 distilbert_mc_classification_openvino
drwxr-xr-x 3 root root      4096 Dec 31 22:07 fields
drwxr-xr-x 2 root root      4096 Dec 31 22:07 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `DistilBertForMultipleChoice` model 😊

In [ ]:
 testing_data = [
            ("In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.",
             "It is eaten with a fork and a knife, It is eaten while held in the hand."),

            ("The Eiffel Tower is located in which country?",
             "Germany, France, Italy"),

            ("Which animal is known as the king of the jungle?",
             "Lion, Elephant, Tiger, Leopard"),

            ("Water boils at what temperature?",
             "90°C, 120°C, 100°C"),

            ("Which planet is known as the Red Planet?",
             "Jupiter, Mars, Venus"),

            ("Which language is primarily spoken in Brazil?",
             "Spanish, Portuguese, English"),

            ("The Great Wall of China was built to protect against invasions from which group?",
             "The Greeks, The Romans, The Mongols, The Persians"),

            ("Which chemical element has the symbol 'O'?",
             "Oxygenm, Osmium, Ozone"),

            ("Which continent is the Sahara Desert located in?",
             "Asia, Africa, South America"),

            ("Which artist painted the Mona Lisa?",
             "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
        ]

In [ ]:
testing_df = spark.createDataFrame(testing_data, ["question", "choices"])
testing_df.show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|question                                                                                  |choices                                                                 |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |
|Which animal is known as the king of the jungle?                                          |Lion, Elephant, Tiger, Leopard                                          |
|Wat

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel

document_assembler = MultiDocumentAssembler() \
            .setInputCols(["question", "choices"]) \
            .setOutputCols(["document_question", "document_choices"])

distilbert_for_multiple_choice = DistilBertForMultipleChoice() \
  .load(f"{MODEL_NAME}_spark_nlp_openvino") \
  .setInputCols(["document_question", "document_choices"])\
  .setOutputCol("answer") \
  .setBatchSize(4)

pipeline = Pipeline(stages=[document_assembler, distilbert_for_multiple_choice])
pipeline_model = pipeline.fit(testing_df)

pipeline_df = pipeline_model.transform(testing_df)

pipeline_df.select("answer").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------+
|answer                                                                                                     |
+-----------------------------------------------------------------------------------------------------------+
|[{chunk, 0, 34, It is eaten with a fork and a knife, {sentence -> 0, chunk -> 0, score -> 0.60487646}, []}]|
|[{chunk, 0, 6, Germany, {sentence -> 0, chunk -> 0, score -> 0.39134768}, []}]                             |
|[{chunk, 0, 5,  Tiger, {sentence -> 0, chunk -> 0, score -> 0.28978878}, []}]                              |
|[{chunk, 0, 3, 90°C, {sentence -> 0, chunk -> 0, score -> 0.35916173}, []}]                                |
|[{chunk, 0, 6, Jupiter, {sentence -> 0, chunk -> 0, score -> 0.35947314}, []}]                             |
|[{chunk, 0, 7,  English, {sentence -> 0, chunk -> 0, score -> 0.36399662}, []}]                            |
|[{chunk, 

That's it! You can now go wild and use hundreds of `DistilBertForMultipleChoice` models from HuggingFace 🤗 in Spark NLP 🚀
